In [ ]:
import numpy as np
import pandas as pd


from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import requests
from datetime import datetime



In [2]:
business_df = pd.read_json("./data/yelp_academic_dataset_business.json", lines=True)

In [3]:
business_df.shape

(150346, 14)

In [5]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [6]:
business_df["state"].value_counts()

state
PA     34039
FL     26330
TN     12056
IN     11247
MO     10913
LA      9924
AZ      9912
NJ      8536
NV      7715
AB      5573
CA      5203
ID      4467
DE      2265
IL      2145
TX         4
CO         3
WA         2
HI         2
MA         2
NC         1
UT         1
MT         1
MI         1
SD         1
XMS        1
VI         1
VT         1
Name: count, dtype: int64

In [9]:
pop_df = pd.read_csv("./data/cbsa-est2019-alldata.csv", encoding='latin1')

In [ ]:
pop_df.head()

,CBSA,MDIV,STCOU,NAME,LSAD,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019
0,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,165252,165252,165585,166634,167442,...,-5,-3,-9,23,-22,-17,-4,-5,-4,-9
1,10180,NaN,48059.0,"Callahan County, TX",County or equivalent,13544,13545,13512,13511,13488,...,-1,-1,-1,-2,-2,-1,0,0,0,-1
2,10180,NaN,48253.0,"Jones County, TX",County or equivalent,20202,20192,20238,20270,19870,...,1,3,14,5,1,1,-1,0,-1,-2
3,10180,NaN,48441.0,"Taylor County, TX",County or equivalent,131506,131515,131835,132853,134084,...,-5,-5,-22,20,-21,-17,-3,-5,-3,-6
4,10420,NaN,NaN,"Akron, OH",Metropolitan Statistical Area,703200,703196,703031,703200,702109,...,-33,-12,-48,-87,-115,-48,-17,-27,-18,-22


In [11]:
pop_df.shape

(2797, 97)

In [26]:
search_strings = ['Philadelphia-Camden', 'Tampa-St.', 'Nashville', 'Indianapolis', 'St. Louis, MO-IL', 'New Orleans', 'Tucson', 'Reno, NV', 'Santa Maria-Santa Barbara', 'Boise City']
pattern = '|'.join(search_strings)

# Filter for exact matches
df_metros = pop_df[pop_df['NAME'].str.contains(pattern, case=False, na=False)]

In [27]:
df_metros

,CBSA,MDIV,STCOU,NAME,LSAD,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019
183,14260,NaN,NaN,"Boise City, ID",Metropolitan Statistical Area,616561,616566,617866,627670,637425,...,-42,15,66,82,159,19,46,63,67,85
669,26900,NaN,NaN,"Indianapolis-Carmel-Anderson, IN",Metropolitan Statistical Area,1887877,1888075,1892640,1910719,1929678,...,22,0,-108,-17,-228,-87,-36,-52,-58,-43
988,34980,NaN,NaN,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area,1646200,1646183,1650831,1674534,1703609,...,8,60,428,326,431,271,101,36,-12,-14
1008,35380,NaN,NaN,"New Orleans-Metairie, LA",Metropolitan Statistical Area,1189866,1189891,1195324,1214008,1226887,...,395,111,279,262,63,27,-35,-31,-29,-43
1112,37980,NaN,NaN,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,5965343,5965677,5970899,5996232,6016616,...,-481,-134,-776,-415,-538,-317,-129,-253,-234,-322
1195,39900,NaN,NaN,"Reno, NV",Metropolitan Statistical Area,425417,425442,425951,427901,431208,...,-37,-11,-16,-26,38,29,25,35,40,45
1263,41180,NaN,NaN,"St. Louis, MO-IL",Metropolitan Statistical Area,2787701,2787751,2789989,2795155,2796682,...,-108,-26,-150,-96,-132,-133,-4,-46,-33,-53
1325,42200,NaN,NaN,"Santa Maria-Santa Barbara, CA",Metropolitan Statistical Area,423895,423947,424231,425453,429699,...,9,4,-67,-35,-84,-38,-8,-13,-2,-12
1410,45300,NaN,NaN,"Tampa-St. Petersburg-Clearwater, FL",Metropolitan Statistical Area,2783243,2783485,2788366,2813269,2843636,...,-111,-40,107,378,641,410,169,164,86,75
1440,46060,NaN,NaN,"Tucson, AZ",Metropolitan Statistical Area,980263,980263,981620,988381,993052,...,-114,-33,-208,-153,-227,-112,-46,-12,15,28


In [37]:
# take just the needed columns
df_metros = df_metros.iloc[:, 3:17]

In [41]:
df_metros.columns[3:14]

Index(['ESTIMATESBASE2010', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'POPESTIMATE2019'],
      dtype='object')

In [58]:
# rename the columns, stripping just the year
columns_to_rename = df_metros.columns[3:14]

# Create a mapping for just those columns
rename_dict = {col: col[-4:] for col in columns_to_rename}
df_metros = df_metros.rename(columns=rename_dict)

#rename the  city names column
df_metros = df_metros.rename(columns={'NAME': 'metro'})

In [51]:
# drop more unnecessary columns
df_metros.drop(columns=['LSAD', 'CENSUS2010POP'], inplace=True)

In [ ]:
# change the name of the metro areas
df_metros['NAME'] = df_metros['NAME'].str.split(r'[,\s-]').str[0]

In [55]:
df_metros['NAME'] = df_metros['NAME'].replace({'New': 'New Orleans', 'St.': 'St. Louis', 'Santa': 'Santa Barbara'})

In [56]:
df_metros

,NAME,2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
183,Boise,616566,617866,627670,637425,649224,662624,674640,691285,709682,728911,749202
669,Indianapolis,1888075,1892640,1910719,1929678,1953600,1971006,1986119,2006760,2027584,2052368,2074537
988,Nashville,1646183,1650831,1674534,1703609,1735012,1768350,1804671,1841294,1874515,1904726,1934317
1008,New Orleans,1189891,1195324,1214008,1226887,1239584,1248905,1260281,1267613,1270326,1270133,1270530
1112,Philadelphia,5965677,5970899,5996232,6016616,6030134,6046382,6056816,6066661,6078522,6091208,6102434
1195,Reno,425442,425951,427901,431208,434878,440110,446329,453709,460605,468335,475642
1263,St. Louis,2787751,2789989,2795155,2796682,2799472,2803430,2807503,2805502,2805758,2803958,2803228
1325,Santa Barbara,423947,424231,425453,429699,434601,439045,442229,444340,445298,445780,446499
1410,Tampa,2783485,2788366,2813269,2843636,2880536,2927395,2983928,3048963,3106922,3154649,3194831
1440,Tucson,980263,981620,988381,993052,997127,1004229,1009103,1016707,1026391,1036554,1047279


In [60]:
# melt the table
df_long = df_metros.melt(id_vars='metro', var_name='Year', value_name='Population')
print(df_long)

             metro  Year  Population
0            Boise  2010      616566
1     Indianapolis  2010     1888075
2        Nashville  2010     1646183
3      New Orleans  2010     1189891
4     Philadelphia  2010     5965677
..             ...   ...         ...
105           Reno  2019      475642
106      St. Louis  2019     2803228
107  Santa Barbara  2019      446499
108          Tampa  2019     3194831
109         Tucson  2019     1047279

[110 rows x 3 columns]
